### Project 2 Example Notebook

In [1]:
## Loading necessary packages

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.datasets import make_classification

#### Notebook Purpose

This notebook covers the basic skills needed for Project 2

1. Ingest csv data
2. Basic data processing
3. Train model
5. Assess model on known labels
6. Score test rows

#### Core data processing

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head()

,creation_date,zip,ward,police_district,community_area,latitude,longitude,ssa,current_activity,number_of_potholes_filled_on_block,completed_within_7_days,request_id
0,2012-03-02,60656.0,41.0,16.0,10.0,41.975,-87.795,NaN,Dispatch Crew,5.0,1,c6fb34de-4e79-4e83-ac3b-cbbf31d6d834
1,2016-03-31,60659.0,50.0,24.0,2.0,41.995,-87.685,NaN,Final Outcome,4.0,1,a50411a7-2c2c-439c-8cd1-62cf18fc2d4a
2,2018-02-27,60634.0,36.0,25.0,19.0,41.937,-87.788,NaN,Final Outcome,76.0,1,55eb1b82-ae52-4e1a-9476-169a59293cf0
3,2011-04-06,60632.0,14.0,9.0,63.0,41.794,-87.688,NaN,Dispatch Crew,0.0,1,59a51854-7646-40d5-a30f-04306a6f9d2f
4,2018-03-28,60643.0,34.0,22.0,75.0,41.680,-87.669,NaN,NaN,NaN,1,06dbfc49-3118-4849-ad24-5d2fd3b1760b


In [3]:
df_test = pd.read_csv('test.csv')
df_test.head()

,creation_date,zip,ward,police_district,community_area,latitude,longitude,ssa,current_activity,number_of_potholes_filled_on_block,request_id
0,2017-02-06,60625.0,40.0,20.0,4.0,41.981,-87.689,NaN,NaN,NaN,b98b4a12-1e86-4634-b7e6-8a8d4c687ef6
1,2012-04-13,60637.0,5.0,3.0,42.0,41.779,-87.585,NaN,Dispatch Crew,86.0,90ad3297-203b-4636-81fa-26e279a323e6
2,2011-08-11,60625.0,39.0,17.0,13.0,41.979,-87.713,NaN,Dispatch Crew,14.0,c5e678b6-5fe5-4bc5-97ed-a9df3ba7f197
3,2014-04-29,60642.0,1.0,14.0,24.0,41.905,-87.666,33.0,NaN,NaN,734f19b6-1a03-40f7-9aa0-2a575ff65fcc
4,2012-08-02,60637.0,5.0,3.0,42.0,41.782,-87.585,NaN,Final Outcome,11.0,f9cf7e4f-5745-4685-8378-03ee40040cc8


In [4]:
#### concatenating the two datasets together so that they get the same data processing before modeling 

df_full = pd.concat([df_train, df_test], ignore_index=True)

In [5]:
### Quick function to pull column types and null counts - keep an eye on column types when modeling

def count_distinct_values(df, include_nulls=False):

    results = []
    
    for column in df.columns:
        distinct_count = df[column].nunique(dropna=not include_nulls)
        total_count = len(df)
        null_count = df[column].isnull().sum()
        
        results.append({
            'Column': column,
            'Data_Type': df[column].dtype,
            'Distinct_Count': distinct_count,
            'Total_Rows': total_count,
            'Null_Count': null_count,
            'Pct_Unique': round(distinct_count / total_count * 100, 2)
        })
    
    return pd.DataFrame(results).sort_values('Distinct_Count', ascending=False)

result = count_distinct_values(df_full)
result.head(len(result))

,Column,Data_Type,Distinct_Count,Total_Rows,Null_Count,Pct_Unique
11,request_id,object,559091,559091,0,100.00
0,creation_date,object,3032,559091,0,0.54
5,latitude,float64,380,559091,871,0.07
6,longitude,float64,370,559091,871,0.07
9,number_of_potholes_filled_on_block,float64,269,559091,161175,0.05
4,community_area,float64,78,559091,331,0.01
1,zip,float64,64,559091,2314,0.01
7,ssa,float64,53,559091,471719,0.01
2,ward,float64,51,559091,331,0.01
3,police_district,float64,27,559091,332,0.00


In [6]:
#### At minimum, let's some basic ID columns as the correct type. 

df_full['zip'] = df_full['zip'].apply(lambda x: str(int(x)).zfill(5) if pd.notna(x) else None) 
df_full['ward'] = df_full['ward'].apply(lambda x: str(int(x)) if pd.notna(x) else None)
df_full['ssa'] = df_full['ssa'].apply(lambda x: str(int(x)) if pd.notna(x) else None)
df_full['police_district'] = df_full['police_district'].apply(lambda x: str(int(x)) if pd.notna(x) else None)
df_full['community_area'] = df_full['community_area'].apply(lambda x: str(int(x)) if pd.notna(x) else None)

    ### you can just run the above function again to check your work. 

#### Join with other provided data

In [7]:
df_congestion = pd.read_csv('congestion_region.csv')
df_acs = pd.read_csv('community_acs.csv')

In [8]:
df_congestion.head()

,region_id,region,min_speed,max_speed,avg_speed,median_speed
0,1,Rogers Park - West Ridge,0,37.88,19.962090,22.50
1,2,Far North West,0,109.43,26.106213,27.27
2,3,North Park-Albany-Linconl Sq,0,97.13,23.521621,23.18
3,4,Edge Water-Uptown,0,73.50,20.191830,20.45
4,5,Dunning-Portage-Belmont Cragn,0,142.50,22.779446,24.55


In [9]:
df_acs.head()

,community_area,under_25_000,_25_000_to_49_999,_50_000_to_74_999,_75_000_to_125_000,_125_000,male_0_to_17,male_18_to_24,male_25_to_34,male_35_to_49,...,white,black_or_african_american,american_indian_or_alaska,asian,native_hawaiin_or_pacific,other_race,multiracial,white_not_hispanic_or_latino,hispanic_or_latino,record_id
0,ALBANY PARK,1269,1916,1801,2306,3379,4799,2955,4513,5442,...,21496,2228,759,7124,1,7888,8334,16115,21108,2023_ALBANY PARK
1,ARCHER HEIGHTS,223,752,441,795,739,1927,732,1102,1240,...,6232,10,108,679,0,3705,3142,2043,11097,2023_ARCHER HEIGHTS
2,ARMOUR SQUARE,701,798,370,637,597,1300,487,871,1174,...,2556,1487,107,8402,61,212,325,2226,565,2023_ARMOUR SQUARE
3,ASHBURN,797,1351,1985,3014,2735,5150,1964,2881,4178,...,11297,18124,697,436,0,7772,4517,3774,19917,2023_ASHBURN
4,AUBURN GRESHAM,2541,2451,1592,2202,1850,5803,1836,2964,3431,...,760,43414,119,399,0,993,798,491,1577,2023_AUBURN GRESHAM


In [10]:
df_crosswalk = pd.read_csv('community_region_crosswalk.csv')
df_crosswalk.head()

,community_area_id,community_area_name,region_id,region_name
0,1,ROGERS PARK,1,Rogers Park - West Ridge
1,2,WEST RIDGE,1,Rogers Park - West Ridge
2,3,UPTOWN,4,Edge Water-Uptown
3,4,LINCOLN SQUARE,3,North Park-Albany-Linconl Sq
4,5,NORTH CENTER,6,Irving Park-Avondale-North Ctr


In [11]:
result = count_distinct_values(df_crosswalk)
result.head(len(result))

,Column,Data_Type,Distinct_Count,Total_Rows,Null_Count,Pct_Unique
0,community_area_id,int64,77,77,0,100.00
1,community_area_name,object,77,77,0,100.00
2,region_id,int64,27,77,0,35.06
3,region_name,object,27,77,0,35.06


In [12]:
df_crosswalk['community_area_id'] = df_crosswalk['community_area_id'].apply(lambda x: str(int(x)) if pd.notna(x) else None)

In [13]:
### creating single unified dataset

## step 1 - join crosswalk to core data by community area name 

df_wide = pd.merge(df_full, df_crosswalk, left_on = 'community_area', right_on ='community_area_id', how = 'left')
df_wide.head()

,creation_date,zip,ward,police_district,community_area,latitude,longitude,ssa,current_activity,number_of_potholes_filled_on_block,completed_within_7_days,request_id,community_area_id,community_area_name,region_id,region_name
0,2012-03-02,60656,41,16,10,41.975,-87.795,None,Dispatch Crew,5.0,1.0,c6fb34de-4e79-4e83-ac3b-cbbf31d6d834,10,NORWOOD PARK,2.0,Far North West
1,2016-03-31,60659,50,24,2,41.995,-87.685,None,Final Outcome,4.0,1.0,a50411a7-2c2c-439c-8cd1-62cf18fc2d4a,2,WEST RIDGE,1.0,Rogers Park - West Ridge
2,2018-02-27,60634,36,25,19,41.937,-87.788,None,Final Outcome,76.0,1.0,55eb1b82-ae52-4e1a-9476-169a59293cf0,19,BELMONT CRAGIN,5.0,Dunning-Portage-Belmont Cragn
3,2011-04-06,60632,14,9,63,41.794,-87.688,None,Dispatch Crew,0.0,1.0,59a51854-7646-40d5-a30f-04306a6f9d2f,63,GAGE PARK,17.0,Midway-Garfield Rdg-Clearing
4,2018-03-28,60643,34,22,75,41.680,-87.669,None,NaN,NaN,1.0,06dbfc49-3118-4849-ad24-5d2fd3b1760b,75,MORGAN PARK,25.0,Beverly-Mt Greenwood-Morgan Park


In [14]:
df_wide = pd.merge(df_wide,df_acs,left_on='community_area_name',right_on='community_area',how='left')
df_wide.head()

,creation_date,zip,ward,police_district,community_area_x,latitude,longitude,ssa,current_activity,number_of_potholes_filled_on_block,...,white,black_or_african_american,american_indian_or_alaska,asian,native_hawaiin_or_pacific,other_race,multiracial,white_not_hispanic_or_latino,hispanic_or_latino,record_id
0,2012-03-02,60656,41,16,10,41.975,-87.795,None,Dispatch Crew,5.0,...,31029.0,515.0,144.0,2809.0,0.0,1759.0,3819.0,27986.0,7802.0,2023_NORWOOD PARK
1,2016-03-31,60659,50,24,2,41.995,-87.685,None,Final Outcome,4.0,...,36943.0,9310.0,681.0,16696.0,17.0,6958.0,8615.0,31486.0,17531.0,2023_WEST RIDGE
2,2018-02-27,60634,36,25,19,41.937,-87.788,None,Final Outcome,76.0,...,27909.0,3370.0,993.0,1939.0,23.0,25743.0,14011.0,10782.0,57511.0,2023_BELMONT CRAGIN
3,2011-04-06,60632,14,9,63,41.794,-87.688,None,Dispatch Crew,0.0,...,13619.0,1403.0,536.0,264.0,1.0,13063.0,5764.0,995.0,31985.0,2023_GAGE PARK
4,2018-03-28,60643,34,22,75,41.680,-87.669,None,NaN,NaN,...,6807.0,12770.0,53.0,211.0,0.0,335.0,1112.0,6485.0,981.0,2023_MORGAN PARK


In [15]:
df_wide = pd.merge(df_wide,df_congestion,on='region_id',how='left')
df_wide.head()

,creation_date,zip,ward,police_district,community_area_x,latitude,longitude,ssa,current_activity,number_of_potholes_filled_on_block,...,other_race,multiracial,white_not_hispanic_or_latino,hispanic_or_latino,record_id,region,min_speed,max_speed,avg_speed,median_speed
0,2012-03-02,60656,41,16,10,41.975,-87.795,None,Dispatch Crew,5.0,...,1759.0,3819.0,27986.0,7802.0,2023_NORWOOD PARK,Far North West,0.0,109.43,26.106213,27.27
1,2016-03-31,60659,50,24,2,41.995,-87.685,None,Final Outcome,4.0,...,6958.0,8615.0,31486.0,17531.0,2023_WEST RIDGE,Rogers Park - West Ridge,0.0,37.88,19.962090,22.50
2,2018-02-27,60634,36,25,19,41.937,-87.788,None,Final Outcome,76.0,...,25743.0,14011.0,10782.0,57511.0,2023_BELMONT CRAGIN,Dunning-Portage-Belmont Cragn,0.0,142.50,22.779446,24.55
3,2011-04-06,60632,14,9,63,41.794,-87.688,None,Dispatch Crew,0.0,...,13063.0,5764.0,995.0,31985.0,2023_GAGE PARK,Midway-Garfield Rdg-Clearing,0.0,131.42,24.723262,29.32
4,2018-03-28,60643,34,22,75,41.680,-87.669,None,NaN,NaN,...,335.0,1112.0,6485.0,981.0,2023_MORGAN PARK,Beverly-Mt Greenwood-Morgan Park,0.0,71.93,25.096574,29.32


In [16]:
### Splitting the data back into test + train after processing 

df_train_wide = df_wide[df_wide['completed_within_7_days'].notna()]
df_test_wide = df_wide[df_wide['completed_within_7_days'].isna()]

print(df_train_wide.shape)
print(df_test_wide.shape)

(447272, 50)
(111819, 50)


### Basic Model 

This model is just to show the basics of how to train a model in Python. This is not a very good model! Make it better through data cleaning, feature engineering, parameter tuning and pulling down additional features from the portal if you so choose!

In [17]:
## remind ourselves the columns we have, their data types, and null counts

result = count_distinct_values(df_train_wide)
result.head(len(result))

,Column,Data_Type,Distinct_Count,Total_Rows,Null_Count,Pct_Unique
11,request_id,object,447272,447272,0,100.00
0,creation_date,object,3022,447272,0,0.68
5,latitude,float64,380,447272,680,0.08
6,longitude,float64,367,447272,680,0.08
9,number_of_potholes_filled_on_block,float64,258,447272,128989,0.06
4,community_area_x,object,78,447272,263,0.02
21,_125_000,float64,77,447272,1524,0.02
44,record_id,object,77,447272,1524,0.02
42,white_not_hispanic_or_latino,float64,77,447272,1524,0.02
36,black_or_african_american,float64,77,447272,1524,0.02


In [18]:
print(df_train_wide.columns)

Index(['creation_date', 'zip', 'ward', 'police_district', 'community_area_x',
       'latitude', 'longitude', 'ssa', 'current_activity',
       'number_of_potholes_filled_on_block', 'completed_within_7_days',
       'request_id', 'community_area_id', 'community_area_name', 'region_id',
       'region_name', 'community_area_y', 'under_25_000', '_25_000_to_49_999',
       '_50_000_to_74_999', '_75_000_to_125_000', '_125_000', 'male_0_to_17',
       'male_18_to_24', 'male_25_to_34', 'male_35_to_49', 'male_50_to_64',
       'male_65', 'female_0_to_17', 'female_18_to_24', 'female_25_to_34',
       'female_35_to_49', 'female_50_to_64', 'female_65', 'total_population',
       'white', 'black_or_african_american', 'american_indian_or_alaska',
       'asian', 'native_hawaiin_or_pacific', 'other_race', 'multiracial',
       'white_not_hispanic_or_latino', 'hispanic_or_latino', 'record_id',
       'region', 'min_speed', 'max_speed', 'avg_speed', 'median_speed'],
      dtype='object')


In [19]:
## Be careful what columns you include, this is a very basic drop just for demonstration purposes

drop_cols = ['creation_date', 'latitude', 'longitude', 'current_activity','number_of_potholes_filled_on_block',
             'completed_within_7_days','request_id', 'community_area_id', 'community_area_name', 'region_id',
             'region_name', 'community_area_y', 'record_id','region']

X = df_train_wide.drop(columns = drop_cols)
y = df_train_wide['completed_within_7_days']

In [20]:
## Checking my work

print(X.columns)

Index(['zip', 'ward', 'police_district', 'community_area_x', 'ssa',
       'under_25_000', '_25_000_to_49_999', '_50_000_to_74_999',
       '_75_000_to_125_000', '_125_000', 'male_0_to_17', 'male_18_to_24',
       'male_25_to_34', 'male_35_to_49', 'male_50_to_64', 'male_65',
       'female_0_to_17', 'female_18_to_24', 'female_25_to_34',
       'female_35_to_49', 'female_50_to_64', 'female_65', 'total_population',
       'white', 'black_or_african_american', 'american_indian_or_alaska',
       'asian', 'native_hawaiin_or_pacific', 'other_race', 'multiracial',
       'white_not_hispanic_or_latino', 'hispanic_or_latino', 'min_speed',
       'max_speed', 'avg_speed', 'median_speed'],
      dtype='object')


In [21]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)

In [22]:
# Create and train the model
# Random Forest takes almost any column type, this is your warning to check your column types and distros for other algorithms

rf_model = RandomForestClassifier(
    n_estimators=100,      # Number of trees in the forest
    max_depth=10,          # Maximum depth of each tree
    random_state=42
)
rf_model.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric(y

In [23]:
# Make predictions
y_pred_rf = rf_model.predict(X_test)

y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]  # Probability of positive class
auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
print(f"\nAUC: {auc_rf:.4f}")

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"\nAccuracy: {accuracy_rf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))


AUC: 0.6255

Accuracy: 0.5926

Classification Report:
              precision    recall  f1-score   support

         0.0       0.60      0.62      0.61     45935
         1.0       0.58      0.57      0.58     43520

    accuracy                           0.59     89455
   macro avg       0.59      0.59      0.59     89455
weighted avg       0.59      0.59      0.59     89455

Confusion Matrix:
[[28328 17607]
 [18833 24687]]


In [24]:
print(df_test_wide.columns)

Index(['creation_date', 'zip', 'ward', 'police_district', 'community_area_x',
       'latitude', 'longitude', 'ssa', 'current_activity',
       'number_of_potholes_filled_on_block', 'completed_within_7_days',
       'request_id', 'community_area_id', 'community_area_name', 'region_id',
       'region_name', 'community_area_y', 'under_25_000', '_25_000_to_49_999',
       '_50_000_to_74_999', '_75_000_to_125_000', '_125_000', 'male_0_to_17',
       'male_18_to_24', 'male_25_to_34', 'male_35_to_49', 'male_50_to_64',
       'male_65', 'female_0_to_17', 'female_18_to_24', 'female_25_to_34',
       'female_35_to_49', 'female_50_to_64', 'female_65', 'total_population',
       'white', 'black_or_african_american', 'american_indian_or_alaska',
       'asian', 'native_hawaiin_or_pacific', 'other_race', 'multiracial',
       'white_not_hispanic_or_latino', 'hispanic_or_latino', 'record_id',
       'region', 'min_speed', 'max_speed', 'avg_speed', 'median_speed'],
      dtype='object')


#### Scoring Test Rows

In [25]:
df_test_scoring = df_test_wide.drop(columns = drop_cols) ## these are the same columns we dropped from the training data!

In [26]:
test_predictions = rf_model.predict_proba(df_test_scoring)[:, 1] # This gives us the probabilities, not the class labels

test_predictions_df = pd.DataFrame({
    'request_id': df_test_wide['request_id'],
    'predicted_probabilitie': test_predictions
})

test_predictions_df.head()

,request_id,predicted_probabilitie
447272,b98b4a12-1e86-4634-b7e6-8a8d4c687ef6,0.404888
447273,90ad3297-203b-4636-81fa-26e279a323e6,0.552526
447274,c5e678b6-5fe5-4bc5-97ed-a9df3ba7f197,0.374305
447275,734f19b6-1a03-40f7-9aa0-2a575ff65fcc,0.462064
447276,f9cf7e4f-5745-4685-8378-03ee40040cc8,0.552526


In [27]:
test_predictions_df.to_csv('test_predictions.csv',index = True) ## send these to TA for AUC

Good luck! Reach out to Professor Cooman or Maura for help. Start early!!